# Getting started

formulae requires a working Python interpreter (3.7+) and the libraries numpy, scipy and pandas with versions specified in the [requirements.txt](https://github.com/bambinos/formulae/blob/master/requirements.txt) file.

Assuming a standard Python environment is installed on your machine (including pip), the latest release of formulae can be installed in one line using pip:

`pip install formulae`

Alternatively, if you want the development version of the package you can install from GitHub:

`pip install git+https://github.com/bambinos/formulae.git`

## User guide

The main function is `design_matrices()`. It takes a model formula and a pandas DataFrame and returns an object of class `DesignMatrices` that contains information about the response, the common effects, and the group specific effects that can be accessed with the attributes `.response`, `.common`, and `.group` respectively. 


Model formulas are much like the very famous model formulas in the programming language R. The extension to group-specific effects is similar to the extension provided by the [lme4](https://CRAN.R-project.org/package=lme4) package, also in R.

## Setup

In [1]:
import numpy as np
import pandas as pd

from formulae import design_matrices

Let's simulate some data to use throughout examples here. The number of observations isn't too important. We keep it low to understand what is going on more easily. However, we do include both numeric and categoric variables to see how formulae handles these types in different scenarios.

In [2]:
np.random.seed(1111)
SIZE=10
data = pd.DataFrame(
    {
        "y1": np.random.normal(size=SIZE),
        "y2": np.random.choice(["A", "B", "C"], size=SIZE),
        "x": np.random.normal(size=SIZE),
        "z": np.random.choice([1, 2, 3], size=SIZE),
        "g": np.random.choice(["Group 1", "Group 2", "Group 3"], size=SIZE),
    }
)
data

,y1,y2,x,z,g
0,-1.300010,A,1.361511,2,Group 2
1,-1.072989,B,1.379059,2,Group 1
2,0.790199,A,0.666949,1,Group 3
3,-0.878265,B,0.171964,1,Group 2
4,1.501822,B,0.557267,1,Group 2
5,-0.023929,C,0.654634,2,Group 2
6,0.275949,A,0.249460,2,Group 3
7,-0.966393,A,-0.626812,3,Group 1
8,-1.580315,A,0.619571,2,Group 3
9,-0.203675,B,0.338624,3,Group 3


## Creating and accessing design matrices

Let's create a `DesignMatrices` object with a numeric response and a numeric predictor. The first argument we pass is the model formula, and the second is the data frame where we take variables from.

In [3]:
dm = design_matrices("y1 ~ x", data)

Under `dm.common` we find an object of class `CommonEffectsMatrix` which stores the design matrix for the common part of the model, as well as other information about the terms in the matrix.

In [4]:
dm.common

CommonEffectsMatrix(  
  shape: (10, 2),
  terms: {  
    'Intercept': {  
      type: intercept,
      full_names: ['Intercept'],
      cols: slice(0, 1, None)
    },
    'x': {  
      type: numeric,
      full_names: ['x'],
      cols: slice(1, 2, None)
    }
  }
)

Use `.design_matrix` to obtain the design matrix for the common part of the model.

In [5]:
dm.common.design_matrix

array([[ 1.        ,  1.36151086],
       [ 1.        ,  1.37905918],
       [ 1.        ,  0.66694943],
       [ 1.        ,  0.17196362],
       [ 1.        ,  0.55726654],
       [ 1.        ,  0.65463436],
       [ 1.        ,  0.24945962],
       [ 1.        , -0.62681223],
       [ 1.        ,  0.61957115],
       [ 1.        ,  0.33862397]])

Or call the `.as_dataframe()` method to return a data frame.

In [6]:
dm.common.as_dataframe().head()

,Intercept,x
0,1.0,1.361511
1,1.0,1.379059
2,1.0,0.666949
3,1.0,0.171964
4,1.0,0.557267


## Categorical predictors

Variables with categorical type in the pandas data frame are recognized and handled as such in formulae. If there's a numerical variable that should be interpreted as categorical and you don't want to manipulate your data manually, you can wrap the name of the variable with `C()`, like `C(variable)`, and formulae will convert it to categorical type.

Categorical predictors are encoded using reference level encoding (i.e. the first level is taken as baseline). This way, the resulting design matrix is of full-rank.

In [7]:
dm = design_matrices("y1 ~ g", data, )
dm.common.as_dataframe().head()

,Intercept,g[Group 2],g[Group 3]
0,1.0,1.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,1.0
3,1.0,1.0,0.0
4,1.0,1.0,0.0


But if we don't have an intercept term, there's no need to use reference encoding for the categorical variables to avoid linear dependencies between the columns and formulae uses cell-means encoding.

In [8]:
dm = design_matrices("y1 ~ 0 + g", data)
dm.common.as_dataframe().head()

,g[Group 1],g[Group 2],g[Group 3]
0,0,1,0
1,1,0,0
2,0,0,1
3,0,1,0
4,0,1,0


Suppose that `z` actually represents a certain categorization represented by numbers. We can use `"y1 ~ C(z)"` and `z` will be internally interpreted as categorical instead of numeric.

In [9]:
dm = design_matrices("y1 ~ C(z)", data)
dm.common.as_dataframe().head()

,Intercept,C(z)[2],C(z)[3]
0,1.0,1.0,0.0
1,1.0,1.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


By default, the `C()` wrapper takes the first level as the reference level (after being sorted with `sorted()`), but we can override this by passing a list with a custom ordering. 

In [10]:
# 2 is taken as baseline, and then it comes 3 and 1.
lvls = [2, 3, 1]
dm = design_matrices("y1 ~ C(z, levels=lvls)", data)
dm.common.as_dataframe().head()

,Intercept,"C(z, levels = lvls)[3]","C(z, levels = lvls)[1]"
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,1.0
3,1.0,0.0,1.0
4,1.0,0.0,1.0


And finally, `C()` also allows us to convert a variable into a `0-1` variable by passing a reference level. In the next chunk of code we use `C(z, 2)`, which means the new variable will be 1 whenever `z==2`, and 0 otherwise.

In [11]:
dm = design_matrices("y1 ~ C(z, 2)", data)
dm.common.as_dataframe().head()

,Intercept,"C(z, 2)"
0,1.0,1.0
1,1.0,1.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


## Adding interactions

The interaction operator is `:`.

In [12]:
dm = design_matrices("y1 ~ x:z", data)
dm.common.as_dataframe().head()

,Intercept,x:z
0,1.0,2.723022
1,1.0,2.758118
2,1.0,0.666949
3,1.0,0.171964
4,1.0,0.557267


The `*` operator is known as the full interaction operator because `x*z` is equivalent to `x + z + x:z`

In [13]:
dm = design_matrices("y1 ~ x*z", data)
dm.common.as_dataframe().head()

,Intercept,x,z,x:z
0,1.0,1.361511,2.0,2.723022
1,1.0,1.379059,2.0,2.758118
2,1.0,0.666949,1.0,0.666949
3,1.0,0.171964,1.0,0.171964
4,1.0,0.557267,1.0,0.557267


And both interaction operators can be used with mixtures of numerical and categorical variables.

In [14]:
dm = design_matrices("y1 ~ 0 + x*g", data)
dm.common.as_dataframe().head()

,x,g[Group 1],g[Group 2],g[Group 3],x:g[Group 2],x:g[Group 3]
0,1.361511,0.0,1.0,0.0,1.361511,0.000000
1,1.379059,1.0,0.0,0.0,0.000000,0.000000
2,0.666949,0.0,0.0,1.0,0.000000,0.666949
3,0.171964,0.0,1.0,0.0,0.171964,0.000000
4,0.557267,0.0,1.0,0.0,0.557267,0.000000


## Function calls

A term does not need to be just a variable in a pandas data frame. It can also be the result of a function call.

In [15]:
dm = design_matrices("y1 ~ np.exp(x)", data)
dm.common.as_dataframe().head()

,Intercept,np.exp(x)
0,1.0,3.902084
1,1.0,3.971164
2,1.0,1.948285
3,1.0,1.187635
4,1.0,1.745894


Or the result of calling a custom function.

In [16]:
def add(x, how_much):
    return x + how_much

dm = design_matrices("y1 ~ add(x, 10)", data)
dm.common.as_dataframe().head()

,Intercept,"add(x, 10)"
0,1.0,11.361511
1,1.0,11.379059
2,1.0,10.666949
3,1.0,10.171964
4,1.0,10.557267


Or even nested function calls!

In [17]:
dm = design_matrices("y1 ~ add(np.exp(x), 5)", data)
dm.common.as_dataframe().head()

,Intercept,"add(np.exp(x), 5)"
0,1.0,8.902084
1,1.0,8.971164
2,1.0,6.948285
3,1.0,6.187635
4,1.0,6.745894


## Built-in transformations

There are only two built-in transformations (for now!) that are commonly used when doing statistical modeling. These are `center()` and `scale()`. They center and standardize a variable, respectively.

In [18]:
dm = design_matrices("y1 ~ scale(x)", data)
dm.common.as_dataframe().head()

,Intercept,scale(x)
0,1.0,1.495846
1,1.0,1.527691
2,1.0,0.235417
3,1.0,-0.662840
4,1.0,0.036374


In [19]:
# check mean is 0 and std is 1
print(np.std(dm.common.as_dataframe()["scale(x)"]))
print(np.mean(dm.common.as_dataframe()["scale(x)"]))

1.0
-3.8857805861880476e-17


These transformations are known as **stateful transformations** because they remember the original values of parameters involved in the transformation (the mean and the standard deviation). This feature is critical when one wants to derive a design matrix based on new data, as required when evaluating the fit of a model on a new dataset.

### Suppresing the default intercept

By default, formulae includes an intercept in the resulting desing matrix. But this can be ommited. Just add a `0` to the model formula as in the following example. 

In [20]:
dm = design_matrices("y1 ~ 0 + x", data)
dm.common.as_dataframe().head()

,x
0,1.361511
1,1.379059
2,0.666949
3,0.171964
4,0.557267


`"y1 ~ -1 + x"` and `"y1 ~ x - 1"` are equivalent alternatives.

## Operations between terms

It is possible someone wants to use a term that is the result of adding two other terms, such as `x + z`. However, if we do `"y ~ x + z"`, formulae will understand we want a model with two predictor terms, `x`, and `z`. Fortunately, there are two (equivalent) ways of telling formulae that we want a term that is the result of performing operations between other terms. This can be achieved either by using the `I()` wrapper or its shorthand `{}`.

In [21]:
dm = design_matrices("y1 ~ I(x + z)", data)
dm.common.as_dataframe().head()

,Intercept,I(x + z)
0,1.0,3.361511
1,1.0,3.379059
2,1.0,1.666949
3,1.0,1.171964
4,1.0,1.557267


In [22]:
dm = design_matrices("y1 ~ {x / z}", data)
dm.common.as_dataframe().head()

,Intercept,I(x / z)
0,1.0,0.680755
1,1.0,0.689530
2,1.0,0.666949
3,1.0,0.171964
4,1.0,0.557267


As you may have noticed, the output shows `I()` instead of `{}`. That's because `{}` is translated into `I()` internally.

## Categorical responses

Currently, formulae converts categorical responses into a binary representation. If the LHS of the formula is a categorical variable with more than two levels, the first level will be represented by a 1, and the rest will be represented by a 0.

Recall `y2` is a categorical variable with levels `A`, `B` and `C`. In this case, formulae will interpret `A` as the reference class and it will be represented by a 1. All the rest are represented with a 0.

In [23]:
dm = design_matrices("y2 ~ x", data)
dm.response

ResponseVector(  
  name: y2,
  type: categoric,
  length: 10,
  refclass: A
)

In [24]:
dm.response.design_vector

array([[1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0]])

But it is also possible to override the default behavior and indicate the reference level with some syntax sugar. If you say `response['level']`, formulae will take level `'level'` as reference. For example

In [25]:
dm = design_matrices("y2['B'] ~ x", data)
dm.response

ResponseVector(  
  name: y2,
  type: categoric,
  length: 10,
  refclass: B
)

In [26]:
# 'y2' is equal to B in four cases
dm.response.design_vector

array([[0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1]])

## Group-specific effects

formulae also handles group-specific effects. They are specified using the pipe operator `|`, such as in [lme4](https://CRAN.R-project.org/package=lme4) package from R. This terms are of the form `(expr|factor)` where the `expr` represents the effect and `factor` the group. [This vignette](https://cran.r-project.org/web/packages/lme4/vignettes/lmer.pdf) from lme4 provides much better information on how this notation works. In a nutshell, group-specific intercepts are specified via `(1|group)` and group-specific slopes are specified via `(variable|group)`. The latter also adds a group-specific intercept by default. This can be overriden doing `(0 + variable|group)`.

First, let's see a formula specifying a common predictor, `x`, and a group-specific intercept for each level of `g`.

In [27]:
dm = design_matrices("y1 ~ x + (1|g)", data)
dm.group

GroupEffectsMatrix(  
  shape: (10, 3),
  terms: {  
    '1|g': {  
      type: intercept,
      groups: ['Group 1', 'Group 2', 'Group 3'],
      full_names: ['1|g[Group 1]', '1|g[Group 2]', '1|g[Group 3]'],
      cols: slice(0, 3, None)
    }
  }
)

In [28]:
dm.common.as_dataframe()

,Intercept,x
0,1.0,1.361511
1,1.0,1.379059
2,1.0,0.666949
3,1.0,0.171964
4,1.0,0.557267
5,1.0,0.654634
6,1.0,0.249460
7,1.0,-0.626812
8,1.0,0.619571
9,1.0,0.338624


In [29]:
dm.group.design_matrix

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

If we now use `(x|g)` we get both the group-specific slope, as well as the intercept, because it is incldued by default.

In [30]:
dm = design_matrices("y1 ~ x + (x|g)", data)
dm.group

GroupEffectsMatrix(  
  shape: (10, 6),
  terms: {  
    '1|g': {  
      type: intercept,
      groups: ['Group 1', 'Group 2', 'Group 3'],
      full_names: ['1|g[Group 1]', '1|g[Group 2]', '1|g[Group 3]'],
      cols: slice(0, 3, None)
    },
    'x|g': {  
      type: numeric,
      groups: ['Group 1', 'Group 2', 'Group 3'],
      full_names: ['x|g[Group 1]', 'x|g[Group 2]', 'x|g[Group 3]'],
      cols: slice(3, 6, None)
    }
  }
)

We can access the different sub-matrices corresponding to each term as follows

In [31]:
dm.group["1|g"] # same as before

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [32]:
dm.group["x|g"]

array([[ 0.        ,  1.36151086,  0.        ],
       [ 1.37905918,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.66694943],
       [ 0.        ,  0.17196362,  0.        ],
       [ 0.        ,  0.55726654,  0.        ],
       [ 0.        ,  0.65463436,  0.        ],
       [ 0.        ,  0.        ,  0.24945962],
       [-0.62681223, -0.        , -0.        ],
       [ 0.        ,  0.        ,  0.61957115],
       [ 0.        ,  0.        ,  0.33862397]])

But as with the common part, this default intercept can be removed

In [33]:
dm = design_matrices("y1 ~ x + (0 + x|g)", data)
dm.group # note there is no 1|x in the terms

GroupEffectsMatrix(  
  shape: (10, 3),
  terms: {  
    'x|g': {  
      type: numeric,
      groups: ['Group 1', 'Group 2', 'Group 3'],
      full_names: ['x|g[Group 1]', 'x|g[Group 2]', 'x|g[Group 3]'],
      cols: slice(0, 3, None)
    }
  }
)

As a final remark, we note that the `expr` part of a group-specific can also be a categorical variable or an interaction. Similar rules apply for the `factor` part.

## Evaluating new data

Both `CommonEffectsMatrix` and `GroupEffectsMatrix` are provided with a method that receives a data frame, with the same structure that the original data frame, and returns a new design matrix.

Suppose we have measured three variables, `y`, `x` and `g` on some subjects. `y` and `x` are numerics, while `g` is a categorical variable that represents some grouping. We also have a model with a common slope, and both varying intercepts and slopes, where we want to standardize the numeric predictor before entering the model. The formula would be `y ~ scale(x) + (scale(x)|g)`.

In [34]:
data = pd.DataFrame({
    "y": np.random.normal(size=50),
    "x": np.random.normal(loc=200, scale=20, size=50),
    "g": np.random.choice(["A", "B", "C"], size=50)
})

In [35]:
dm = design_matrices("y ~ scale(x) + (scale(x)|g)", data)
print(dm.common)

CommonEffectsMatrix(  
  shape: (50, 2),
  terms: {  
    'Intercept': {  
      type: intercept,
      full_names: ['Intercept'],
      cols: slice(0, 1, None)
    },
    'scale(x)': {  
      type: numeric,
      full_names: ['scale(x)'],
      cols: slice(1, 2, None)
    }
  }
)


In [36]:
print(dm.group)

GroupEffectsMatrix(  
  shape: (50, 6),
  terms: {  
    '1|g': {  
      type: intercept,
      groups: ['A', 'B', 'C'],
      full_names: ['1|g[A]', '1|g[B]', '1|g[C]'],
      cols: slice(0, 3, None)
    },
    'scale(x)|g': {  
      type: numeric,
      groups: ['A', 'B', 'C'],
      full_names: ['scale(x)|g[A]', 'scale(x)|g[B]', 'scale(x)|g[C]'],
      cols: slice(3, 6, None)
    }
  }
)


And now suppose we have new data for a set of new subjects. We can derive both common and group-specific design matrices for these new individuals without having to specify the model again. We can instead use the `._evaluate_new_data()` method. This method takes a data frame and returns either a new `CommonEffectsMatrix` or a new `GroupEffectsMatrix` depending on which object the method was called.

In [37]:
data2 = pd.DataFrame({
    "x": np.random.normal(loc=180, scale=20,size=10),
    "g": np.random.choice(["A", "B", "C"], size=10)
})

In [38]:
common_new = dm.common._evaluate_new_data(data2)
common_new

CommonEffectsMatrix(  
  shape: (10, 2),
  terms: {  
    'Intercept': {  
      type: intercept,
      full_names: ['Intercept'],
      cols: slice(0, 1, None)
    },
    'scale(x)': {  
      type: numeric,
      full_names: ['scale(x)'],
      cols: slice(1, 2, None)
    }
  }
)

And if we explore the design matrix for the common terms we notice the scaling of the new data used the mean and standard deviation `x` in the first dataset.

In [39]:
common_new.as_dataframe()

,Intercept,scale(x)
0,1.0,0.093144
1,1.0,-0.792722
2,1.0,-1.275461
3,1.0,-1.891837
4,1.0,-1.410180
5,1.0,-1.762914
6,1.0,0.065393
7,1.0,-3.441713
8,1.0,-2.492593
9,1.0,-3.075939


Something similar can be done with the group-specific part.

In [40]:
group_new = dm.group._evaluate_new_data(data2)
group_new

GroupEffectsMatrix(  
  shape: (10, 6),
  terms: {  
    '1|g': {  
      type: intercept,
      groups: ['A', 'B', 'C'],
      full_names: ['1|g[A]', '1|g[B]', '1|g[C]'],
      cols: slice(0, 3, None)
    },
    'scale(x)|g': {  
      type: numeric,
      groups: ['A', 'B', 'C'],
      full_names: ['scale(x)|g[A]', 'scale(x)|g[B]', 'scale(x)|g[C]'],
      cols: slice(3, 6, None)
    }
  }
)

In [41]:
group_new["scale(x)|g"]

array([[ 0.        ,  0.        ,  0.09314421],
       [-0.        , -0.        , -0.7927217 ],
       [-0.        , -0.        , -1.27546133],
       [-0.        , -0.        , -1.89183665],
       [-0.        , -0.        , -1.41018013],
       [-1.76291403, -0.        , -0.        ],
       [ 0.        ,  0.        ,  0.06539344],
       [-0.        , -3.44171315, -0.        ],
       [-0.        , -0.        , -2.49259338],
       [-0.        , -0.        , -3.07593863]])

Why the first column of zeros? Well, it looks like there's nobody in the new data has `g == "A"`

In [42]:
data2["g"].values

array(['C', 'C', 'C', 'C', 'C', 'A', 'C', 'B', 'C', 'C'], dtype=object)